In [1]:
##########################################
#Step 0: Import libraries
##########################################

In [2]:
#generic data analysis 
import os
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno

In [3]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Reporting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

#metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [4]:
##########################################
#Step 1: Import data
##########################################

In [5]:
#Setting directory
directory = r'C:\Users\jlenehan\OneDrive - Intel Corporation\Documents\0 - Data Science\Data Analytics Essentials\UCDPA_JohnLenehan\UCDPA_JohnLenehan'
os.chdir(directory)

In [6]:
#importing car crash data from chicago data portal
#url to overview page - https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
collision_json = r'https://data.cityofchicago.org/resource/85ca-t3if.json?$limit=1000000' #json url

#using read_json function to read in dataset
collision_raw = pd.read_json(collision_json) #reading collisions json

In [7]:
print(collision_raw.head())

                                     crash_record_id               crash_date  \
0  25d92973475a04a93e7fd206fbfce57e8a9a1e25cc85a7...  2023-05-16T23:12:00.000   
1  375ac7f6fcb4ef73d728edc52ed556f23fd465a351833f...  2023-05-16T23:06:00.000   
2  246fea010af2010860046c6ef36efb75a8c60244088939...  2023-05-16T23:05:00.000   
3  18c220f7eeceb2cf6f9512c9b83382da28d8565fbbaaec...  2023-05-16T22:20:00.000   
4  cfecdce601503162eb09337bd6051ea358dca7294d440b...  2023-05-16T21:45:00.000   

   posted_speed_limit traffic_control_device      device_condition  \
0                  30         TRAFFIC SIGNAL  FUNCTIONING PROPERLY   
1                  30            NO CONTROLS           NO CONTROLS   
2                  30            NO CONTROLS           NO CONTROLS   
3                  25            NO CONTROLS           NO CONTROLS   
4                  30                UNKNOWN  FUNCTIONING PROPERLY   

  weather_condition      lighting_condition      first_crash_type  \
0             CLEAR  DA

In [8]:
#importing beat data to join to main dataset
#source - Chicago Data Portal
#beat data url - https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-current-/aerh-rz74
beat_data=pd.read_csv('PoliceBeatDec2012.csv')

print(beat_data.info())
print(beat_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   the_geom  277 non-null    object
 1   DISTRICT  277 non-null    int64 
 2   SECTOR    277 non-null    int64 
 3   BEAT      277 non-null    int64 
 4   BEAT_NUM  277 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 10.9+ KB
None
         DISTRICT      SECTOR        BEAT     BEAT_NUM
count  277.000000  277.000000  277.000000   277.000000
mean    12.122744    2.028881    2.028881  1235.241877
std      7.232131    0.924249    0.924249   722.945828
min      1.000000    0.000000    0.000000   111.000000
25%      6.000000    1.000000    1.000000   633.000000
50%     11.000000    2.000000    2.000000  1131.000000
75%     18.000000    3.000000    3.000000  1813.000000
max     31.000000    5.000000    5.000000  3100.000000


In [9]:
##########################################
#Step 2: Merge Data
##########################################

In [10]:
#joining collision data to beat data - inner join
collisions = collision_raw.merge(beat_data, how='inner',
                                 left_on='beat_of_occurrence',
                                 right_on='BEAT_NUM'
                                 )

print(collisions.head())

                                     crash_record_id               crash_date  \
0  25d92973475a04a93e7fd206fbfce57e8a9a1e25cc85a7...  2023-05-16T23:12:00.000   
1  9cc82b5ba91f3da11c5bc561a8cfc090b30424ede46fec...  2023-05-16T15:00:00.000   
2  2a5a2c1d2620e5f81362d99d4956a80884413241471bb7...  2023-05-15T20:40:00.000   
3  caf58dafbea706f6d18c6016ca7aee6443db31f285b662...  2023-05-14T17:07:00.000   
4  49ddea33f1e483cffcbb1c2595527f7a3ec1484d14a9d8...  2023-05-14T16:00:00.000   

   posted_speed_limit traffic_control_device      device_condition  \
0                  30         TRAFFIC SIGNAL  FUNCTIONING PROPERLY   
1                  30            NO CONTROLS           NO CONTROLS   
2                  30            NO CONTROLS           NO CONTROLS   
3                  30         TRAFFIC SIGNAL  FUNCTIONING PROPERLY   
4                  30            NO CONTROLS           NO CONTROLS   

  weather_condition      lighting_condition          first_crash_type  \
0             CLEAR

In [11]:
#alternatively - can get district from first 2 digits of beat of occurrence
collisions['district'] = collisions['beat_of_occurrence'].replace(r'^\d{2}', '')

In [12]:
##########################################
#Step 3: Describe data
##########################################

In [13]:
#Describe recent incidents dataset
print(collisions.columns)
print(collisions.info())
print(collisions.describe())
print(collisions.shape)


Index(['crash_record_id', 'crash_date', 'posted_speed_limit',
       'traffic_control_device', 'device_condition', 'weather_condition',
       'lighting_condition', 'first_crash_type', 'trafficway_type',
       'alignment', 'roadway_surface_cond', 'road_defect', 'report_type',
       'crash_type', 'intersection_related_i', 'damage',
       'date_police_notified', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'beat_of_occurrence', 'num_units', 'most_severe_injury',
       'injuries_total', 'injuries_fatal', 'injuries_incapacitating',
       'injuries_non_incapacitating', 'injuries_reported_not_evident',
       'injuries_no_indication', 'injuries_unknown', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude', 'location',
       'hit_and_run_i', 'statements_taken_i', 'crash_date_est_i',
       'private_property_i', 'photos_taken_i', 'work_zone_i', 'work_zone_type',
       'workers_present_i', 

In [14]:
#defining custom function to print all unique values and their counts from each column
def print_uniques(df):
    #for large datasets - using generator object for speed
    uniques_generator = ((x, df[x].unique(), df[x].nunique()) for x in df.columns)
    
    print('\nUnique Values:')
    for x, unique_values, num_unique in uniques_generator:
        #building in try-except statement for possible strange column data
        try:
            print(f"{x}: \n {unique_values} \n ({num_unique} unique values)")
        except Exception as e:
            print(f"Error occurred in column '{x}': {str(e)}")


In [15]:
#calling custom function on collisions df
print_uniques(collisions)


Unique Values:
crash_record_id: 
 ['25d92973475a04a93e7fd206fbfce57e8a9a1e25cc85a7e998bb71e476a95e2cb27abd1cef40a8efd9ec4929c34da8f7f5403333b420bf4ca753bf77fd8417fb'
 '9cc82b5ba91f3da11c5bc561a8cfc090b30424ede46fecea8758ee2dbf8eef01656378e75c971f01275eb53c4bb60d3b45566107c73c2ad6c6c7cac55f98a221'
 '2a5a2c1d2620e5f81362d99d4956a80884413241471bb70373a0e26d39fe49769d4da334fa05ac12d08531fef35da861a66e6511146f305ddb254ecdd5925b7d'
 ...
 '4ff35be90452c089a764b5fde49353191e266ee9406502c8532562c2975439ff7de1b7d8e8860be634d6ef450639be0c148a619df2d039bff625517be294041d'
 '5c46814b2045698719c88eb26a1c0f4bb8d694fa5412250a5823bf72d63b6480e456d94bdd0197e20da6983d1e32128827301cd0bc15ea60175e41016fcc1360'
 '7e0df28ef853668f5c9b252a651e0dcdb2d6a8626c263cac153b8e1ffded3a74d6f5623549be2de55600dff1389d8f69a907ce8967fae5639cd748012a3999fd'] 
 (722796 unique values)
crash_date: 
 ['2023-05-16T23:12:00.000' '2023-05-16T15:00:00.000'
 '2023-05-15T20:40:00.000' ... '2015-10-17T11:45:00.000'
 '2015-10-13T20:00

most_severe_injury: 
 ['NO INDICATION OF INJURY' 'REPORTED, NOT EVIDENT'
 'NONINCAPACITATING INJURY' 'INCAPACITATING INJURY' 'FATAL' nan] 
 (5 unique values)
injuries_total: 
 [ 0.  2.  1.  4.  3.  5. nan  6.  7.  9. 21.  8. 12. 11. 10. 15. 13. 17.
 16. 19.] 
 (19 unique values)
injuries_fatal: 
 [ 0.  1. nan  3.  2.  4.] 
 (5 unique values)
injuries_incapacitating: 
 [ 0.  1.  2. nan  3.  4.  5.  6.  7.] 
 (8 unique values)
injuries_non_incapacitating: 
 [ 0.  1.  2.  3.  5. nan  4.  6. 19.  9.  7. 12.  8. 10. 14. 21. 16. 15.
 11. 18.] 
 (19 unique values)
injuries_reported_not_evident: 
 [ 0.  2.  1.  3. nan  5.  4.  7.  6.  8.  9. 15. 10. 11.] 
 (13 unique values)
injuries_no_indication: 
 [ 3.  2.  1.  4.  5.  7.  0.  6.  9. nan  8. 10. 13. 16. 31. 37. 12. 36.
 43. 11. 14. 30. 28. 22. 18. 40. 26. 17. 61. 19. 15. 20. 27. 21. 29. 32.
 45. 23. 50. 46. 24. 33. 34. 42. 39. 25. 38.] 
 (46 unique values)
injuries_unknown: 
 [ 0. nan] 
 (1 unique values)
crash_hour: 
 [23 15 20 17 16 21  7

TypeError: unhashable type: 'dict'

In [ ]:
##########################################
#Step 4: Clean + Manipulate data
##########################################

In [ ]:
#Converting Incident datetime, Report Datetime to a datetime object
collisions['crash_date'] = collisions['crash_date'].apply(pd.to_datetime)

#extracting year from datestamp
collisions['crash_year'] = collisions['crash_date'].dt.year

In [ ]:
#Visualising missing data
#Sorting values by report received date
collisions = collisions.sort_values(by='crash_date',ascending=True)

#plotting matrix of missing data
msno.matrix(collisions)
plt.show()

#info of sorted data
print(collisions.info())

In [ ]:
#defining unnecessary columns
drop_cols = ['location', 'crash_date_est_i','report_type', 'intersection_related_i',
       'hit_and_run_i', 'photos_taken_i', 'crash_date_est_i', 'injuries_unknown',
       'private_property_i', 'statements_taken_i', 'dooring_i', 'work_zone_i',
       'work_zone_type', 'workers_present_i','lane_cnt','the_geom','rd_no',
            'SECTOR','BEAT','BEAT_NUM']

#dropping columns
collisions=collisions.drop(columns=drop_cols)

#plotting matrix of missing data
msno.matrix(collisions)
plt.show()

#info of sorted data
print(collisions.info())

In [ ]:
#drop all rows with missing latitude/longitude data
collisions.dropna(subset=['longitude','latitude'],inplace=True)

#replacing all null values in injuries columns with 0
#defining injury columns
num_injury_cols = ['injuries_total','injuries_fatal','injuries_incapacitating',
                  'injuries_non_incapacitating','injuries_reported_not_evident',
                  'injuries_no_indication']
collisions[num_injury_cols] = collisions[num_injury_cols].fillna(0)


In [ ]:
#Some incorrect lat/long data - need to remove these rows
collisions = collisions[collisions['longitude']<-80]
collisions = collisions[collisions['latitude']>40]

In [ ]:
print(collisions.describe())

In [ ]:
##########################################
#Step 5: Plot data
##########################################

In [ ]:
#setting range of plot to be the minimum/maximum values for lat/long

#plotting lat/long data showing locations of crashes
plt.figure(figsize=(20,12))
collisions.plot(kind='scatter',x='longitude',y='latitude',grid=True,legend='casualty',alpha=0.2)

plt.show()

In [ ]:
#plot graphs of fatalities by weather event, roadway condition
roadway_surf_count = collisions['DISTRICT'].value_counts()

roadway_surf_count.plot.bar()
plt.xlabel('District')
plt.ylabel('Crash count')
plt.title('Car Collisions by District')
plt.show()

In [ ]:
#plot graphs of crash type 
crash_type_count = collisions['crash_type'].value_counts()

crash_type_count.plot.bar()
plt.xlabel('Crash Type')
plt.ylabel('Crash count')
plt.title('Car Collisions by Crash Type')
plt.show()

In [ ]:
#plot graphs of fatalities by weather event, roadway condition
injuries_total_count = collisions['injuries_incapacitating'].value_counts()

injuries_total_count

In [ ]:
##########################################
#Step 6: Machine Learning
##########################################

In [ ]:
##Steps

#1. Exploratory Data Analysis - histograms, check for nulls, describe/info etc DONE
#2. Create test set 
#3. Data visualisation DONE
#4. Data preparation (combine some attributes, transform others, clean data, text vs. cat attributes, )
    ##NB - lots of categorical attributes in dataset, need to figure out how to deal with this
#6. Select/train a model (look up classifier algorithms - decision tree/random forest etc)
#7. Evaluate model (look up good evaluation criteria for classification model)
#8. Fine tuning (GridsearchCV/RandomisedSearchCV)
#9. Ensemble methods
#10. Analyse best models and errors
#11. Evaluate on test set

In [ ]:
#plotting histograms of numerical values
collisions.hist(bins=50,figsize=(16,12))
plt.show()

In [ ]:
#4. Data preparation (combine some attributes, transform others, clean data, text vs. cat attributes, )
    ##NB - lots of categorical attributes in dataset, need to figure out how to deal with this
#Note 1 - Latitude and longitude data should be transformed for ML analysis
#

collisions.columns

In [ ]:
ml_cols = ['posted_speed_limit','traffic_control_device', 'device_condition', 'weather_condition',
          'lighting_condition', 'first_crash_type', 'trafficway_type','alignment', 
           'roadway_surface_cond', 'road_defect', 'crash_type','damage','prim_contributory_cause',
          'sec_contributory_cause','street_direction','num_units', 'DISTRICT',
          'crash_hour','crash_day_of_week','latitude', 'longitude']
cat_cols = ['traffic_control_device', 'device_condition', 'weather_condition', 'DISTRICT',
           'lighting_condition', 'first_crash_type', 'trafficway_type','alignment',
           'roadway_surface_cond', 'road_defect', 'crash_type','damage','prim_contributory_cause',
           'sec_contributory_cause','street_direction','num_units']

collisions_ml = collisions[ml_cols].copy()

#encoding categorical values
label_encoder = LabelEncoder()
for col in collisions_ml[cat_cols].columns:
    collisions_ml[col] = label_encoder.fit_transform(collisions_ml[col])


In [ ]:
#scaling latitude and longitude data
scaler = StandardScaler()

# Logarithmic transformation on longitude
collisions_ml['neg_log_longitude'] = scaler.fit_transform(np.log1p(-collisions_ml['longitude']).values.reshape(-1,1))

# Normalisation on latitude
collisions_ml['norm_latitude'] = scaler.fit_transform(collisions['latitude'].values.reshape(-1, 1))

In [ ]:
collisions_ml[['neg_log_longitude','norm_latitude']].hist(bins=50)
plt.show()

In [ ]:
#transforming crash_hour - 
#data is cyclic, can be encoded using trig transforms

#trig transformation - sin(crash_hr)
collisions_ml['sin_hr'] = np.sin(2*np.pi*collisions_ml['crash_hour']/24)


In [ ]:
#drop previous latitude/longitude columns
lat_long_drop_cols = ['longitude','latitude']
collisions_ml.drop(lat_long_drop_cols,axis=1,inplace=True)

#drop crash_hour column
collisions_ml.drop('crash_hour',axis=1,inplace=True)

In [ ]:
collisions_ml['sin_hr'].hist(bins=12)

In [ ]:
#2. Create test set (random or stratified, choose training set size) <--
#setting X and y values

X = collisions_ml.drop('crash_type', axis=1)
y = collisions_ml['crash_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#validation set 
#can use to check how it performs in an entirely separate model
#can use CV instead

In [ ]:
print(collisions_ml.head())

In [ ]:
#6. Select/train a model (look up classifier algorithms - decision tree/random forest etc)


In [ ]:
#Classifier 1 - Random Forest
RFClassifier = RandomForestClassifier(n_estimators=300,min_samples_split=10)

RFClassifier.fit(X_train,y_train)

In [ ]:
#Predictions
#make predictions on the training set
y_train_pred = RFclassifier.predict(X_train)

# Make predictions on the test data
y_test_pred = RFclassifier.predict(X_test)

In [ ]:
#7. Evaluate model (look up good evaluation criteria for classification model)
# Calculate the accuracy of the model

#calculating accuracy of model on training data
train_accuracy = accuracy_score(y_train, y_train_pred)

#calculating accuracy of model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)

#computing f1 score,precision,recall
f1 = f1_score(y_test, y_test_pred)
precision = precision_score(y_test,y_test_pred)
recall = recall_score(y_test,y_test_pred)

#comparing performances
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

#print precision score
print("Precision Score:", precision)

#print recall score
print("Recall Score:", recall)

#print f1 score
print("F1 Score:", f1)


In [ ]:
#Classifier 2 - K Nearest Neighbours
#instantiate KNN Classifier
KNNClassifier = KNeighborsClassifier(n_neighbors=10)

KNNClassifier.fit(X_train,y_train)

In [ ]:
#Predictions
#make predictions on the training set
y_train_pred = KNNClassifier.predict(X_train)

# Make predictions on the test data
y_test_pred = KNNClassifier.predict(X_test)

In [ ]:
#7. Evaluate model
# Calculate the accuracy of the model

#calculating accuracy of model on training data
train_accuracy = accuracy_score(y_train, y_train_pred)

#calculating accuracy of model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)

#computing f1 score,precision,recall
f1 = f1_score(y_test, y_test_pred)
precision = precision_score(y_test,y_test_pred)
recall = recall_score(y_test,y_test_pred)

#comparing performances
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

#print precision score
print("Precision Score:", precision)

#print recall score
print("Recall Score:", recall)

#print f1 score
print("F1 Score:", f1)


In [ ]:
#cross val to determine optimal n_neighbors

# Define the range of n_neighbors values to explore
n_range = range(1, 21)

# Create an empty list to store the mean cross-validation scores
cv_scores = []

# Perform cross-validation for each n_neighbors value
for n in n_range:
    # Create a KNN classifier with n_neighbors value
    knn = KNeighborsClassifier(n_neighbors=n)
    
    # Perform cross-validation using 5-fold CV
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    
    # Compute the mean cross-validation score
    mean_score = np.mean(scores)
    
    # Append the mean score to the cv_scores list
    cv_scores.append(mean_score)

# Find the optimal number of neighbors with the highest score
opt_n = n_range[np.argmax(cv_scores)]

print("Optimal number of neighbors:", opt_n)

In [ ]:
# Plot the accuracy vs. number of neighbors
plt.plot(n_range, cv_scores)
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Number of Neighbors')
plt.show()

In [ ]:
#feature engineering
# Retrieve the feature importances from the trained classifier
importances = RFClassifier.feature_importances_

# Create a DataFrame to display the feature importances
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances_df = feature_importances_df.sort_values('Importance', ascending=False)
print(feature_importances_df)

In [ ]:
#8. Fine tuning (GridsearchCV/RandomisedSearchCV)
# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Create an instance of Randomized Search Cross-Validation
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(), param_distributions=param_grid, cv=5)

# Perform the randomized search on the training data
random_search.fit(X_train, y_train)

# Retrieve the best model and its performance
best_classifier = random_search.best_estimator_
best_accuracy = random_search.best_score_

print("Best Accuracy:", best_accuracy)
print("Best Model:", best_classifier)

In [ ]:
#10. Analyse best models and errors

In [ ]:
#11. Evaluate on test set